In [ ]:
import requests
import json
import openai
from config import *

API_KEY=api_key
BASE_URL="https://api.deepseek.com"


# 测试数据
test_data = [
    {"news_id": "1", "title": "全球经济形势严峻，多个国家迎来经济衰退", "keywords": "经济, 全球, 衰退, 国家"},
    {"news_id": "2", "title": "双十一大促销，商品折扣力度空前", "keywords": "电商, 促销, 双十一, 折扣"},
    {"news_id": "3", "title": "社交平台更新新功能，用户体验提升", "keywords": "社交平台, 更新, 功能, 用户体验"},
    {"news_id": "4", "title": "如何保持健康饮食，专家给出建议", "keywords": "健康, 饮食, 专家, 建议"},
    {"news_id": "5", "title": "世界杯即将开幕，足球迷热议各大球队阵容", "keywords": "世界杯, 足球, 球迷, 阵容"},
    {"news_id": "6", "title": "新型人工智能芯片发布，推动行业发展", "keywords": "人工智能, 芯片, 科技, 行业"}
]

category_list = ["财经", "娱乐", "体育", "科技", "健康", "教育", "军事", "旅游", "汽车", "房产"]

In [7]:

model = openai.OpenAI(
                api_key=API_KEY,
                base_url=BASE_URL,
            )
tools = {
                "type": "function",
                "function": {
                    "name": "classification",
                    "description": "根据给定的新闻标题和关键词，进行新闻分类",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "category": {
                                "type": "string",
                                "description": "新闻分类",
                            },
                        },
                        "required": ["category"],
                    },
                },
            }

In [13]:
for data in test_data:
    temp_str = f"新闻标题：{data['title']}，关键词：{data['keywords']}，请根据关键词进行新闻分类"

    response = model.chat.completions.create(model="deepseek-chat", messages=[{
    "role": "system", "content": f"""
        你是一个专业的新闻分类器，能够快速精简的从提供的文本中提取出新闻分类。
        分类的结果必须是以下列表中的一个：{category_list}。
        注意，你必须调用提供的函数进行信息提取，将提取的信息填入函数的参数中。
        直接返回提取的结果，不要回复多余的内容。
        """},
    {"role": "user", "content": f"""
        请根据以下材料，提取新闻分类：
        {temp_str}
        """},
        ],tools=[tools],tool_choice="required")
        
    tool_call = response.choices[0].message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    category = arguments["category"]
    result = {
        "news_id": data["news_id"],
        "category": category,
    }
    print(result)
    # break

{'news_id': '1', 'category': '财经'}
{'news_id': '2', 'category': '财经'}
{'news_id': '3', 'category': '科技'}
{'news_id': '4', 'category': '健康'}
{'news_id': '5', 'category': '体育'}
{'news_id': '6', 'category': '科技'}
